In [1]:
%%capture
%pip install pandas
%pip install transformers

In [13]:
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, pipeline
import pandas as pd
from tqdm import tqdm

import warnings
warnings.simplefilter('ignore')

### Initialize NLBB-3.3b model

In [3]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
print(device)

cuda:0


In [4]:
# Load the tokenizer and model
model_name = "facebook/nllb-200-distilled-1.3B"

model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [35]:
source_lang = "eng_Latn"
target_lang = "por_Latn"
enpt_pipeline = pipeline('translation', model=model, tokenizer=tokenizer, 
                          src_lang=source_lang, tgt_lang=target_lang,
                          batch_size=64, device=device)

### Read dataset from CSV

In [5]:
df_dev = pd.read_csv('data/dev.csv')
df_train = pd.read_csv('data/train.csv')
df_tst = pd.read_csv('data/tst.csv')

In [6]:
df_dev.head()

,Unnamed: 0,context,question,answerA,answerB,answerC,correct
0,0,Tracy didn't go home that evening and resisted...,What does Tracy need to do before this?,make a new plan,Go home and see Riley,Find somewhere to go,C
1,1,Sydney walked past a homeless woman asking for...,How would you describe Sydney?,sympathetic,like a person who was unable to help,incredulous,A
2,2,Sasha protected the patients' rights by making...,What will patients want to do next?,write new laws,get petitions signed,live longer,B
3,3,Jordan was in charge of taking the food on the...,How would Jordan feel afterwards?,horrible that he let his friends down on the c...,happy that he doesn't need to do the cooking o...,very proud and accomplished about the camping ...,A
4,4,Kendall opened their mouth to speak and what c...,How would you describe Kendall?,a very quiet person,a very passive person,a very aggressive and talkative person,C


#### Test Run

In [7]:
row_index = 0  # select the row index you're interested in
columns_of_interest = ['context', 'question', 'answerA', 'answerB', 'answerC']  # select the columns you want

row_values = df_train.loc[row_index, columns_of_interest].tolist()

In [8]:
row_values

['Even though she had homework to do that night, Jesse helped Skylar study.',
 'What will Jesse want to do next?',
 'read homework to Skylar',
 'help Skylar finish',
 'skip her studying']

In [19]:
enpt_pipeline(row_values)

[{'translation_text': 'Mesmo tendo de fazer o dever de casa naquela noite, Jesse ajudou a Skylar a estudar.'},
 {'translation_text': 'O que é que o Jesse vai querer fazer a seguir?'},
 {'translation_text': 'Leia os trabalhos de casa à Skylar.'},
 {'translation_text': 'Ajudem a Skylar a terminar .'},
 {'translation_text': 'Não a deixes estudar.'}]

In [20]:
def translate(str_list, model):

    translated = []

    out = enpt_pipeline(str_list)
    translated = [d['translation_text'] for d in out]
    
    return translated

In [21]:
out = translate(row_values, model)

In [22]:
out

['Mesmo tendo de fazer o dever de casa naquela noite, Jesse ajudou a Skylar a estudar.',
 'O que é que o Jesse vai querer fazer a seguir?',
 'Leia os trabalhos de casa à Skylar.',
 'Ajudem a Skylar a terminar .',
 'Não a deixes estudar.']

### Translate dev base first

In [23]:
# New dataframe for the translation
df_dev_pt = df_dev.copy()
df_dev_pt.loc[:, ['context', 'question', 'answerA', 'answerB', 'answerC']] = None

In [24]:
df_dev_pt

,Unnamed: 0,context,question,answerA,answerB,answerC,correct
0,0,None,None,None,None,None,C
1,1,None,None,None,None,None,A
2,2,None,None,None,None,None,B
3,3,None,None,None,None,None,A
4,4,None,None,None,None,None,C
...,...,...,...,...,...,...,...
1949,1949,None,None,None,None,None,A
1950,1950,None,None,None,None,None,B
1951,1951,None,None,None,None,None,C
1952,1952,None,None,None,None,None,B


In [27]:
def create_batch(df, step, base):
    row_values = []
    for i in range(step):
        columns_of_interest = ['context', 'question', 'answerA', 'answerB', 'answerC']  # select the columns you want
        row_values += (df.loc[base + i, columns_of_interest].tolist())
    return row_values

In [28]:
batch_size = 1
n_cols = 5

def translate_base(df, df_pt, model):
    for i in tqdm(range(0, len(df), batch_size)):
        
        strings = create_batch(df, batch_size, i)
        translated_strings = translate(strings, model)

        df_pt.iat[i, 1] = translated_strings[0]
        df_pt.iat[i, 2] = translated_strings[1]
        df_pt.iat[i, 3] = translated_strings[2]
        df_pt.iat[i, 4] = translated_strings[3]
        df_pt.iat[i, 5] = translated_strings[4]

In [31]:
translate_base(df_dev, df_dev_pt, model)

100%|██████████| 1954/1954 [19:39<00:00,  1.66it/s]


In [20]:
df_dev_pt

,Unnamed: 0,context,question,answerA,answerB,answerC,correct
0,0,Tracy não ia para casa naquela noite e resisti...,O que é necessário para Tracy antes disso?,elaborar um novo plano,Ir para casa e ver Riley,Encontrar em algum lugar para ir,C
1,1,Sydney passou uma mulher em situação de rua pe...,Como descreveria Sydney?,simpática,como uma pessoa que não conseguiu ajudar,inchaço,A
2,2,Sasha protegeu os direitos dos pacientes ao fa...,O que os pacientes querem fazer a seguir?,redigidos novos diplomas,au,vida,B
3,3,Jordan foi responsável por levar a comida na v...,Como a Jordânia se sentiria depois?,horrível que ele deixe seus amigos para baixo ...,feliz que ele não precisa fazer o cozimento na...,muito orgulho e realizado com relação à viagem...,A
4,4,Kendall abriu a boca para falar e o que saiu c...,Como descrever Kendall?,uma pessoa muito silenciosa,uma pessoa muito passiva,uma pessoa muito agressiva e falante,C
...,...,...,...,...,...,...,...
1949,1949,Bailey destruiu o bunker do inimigo e venceu a...,O que é necessário para Bailey antes disso?,esperam os movimentos de tropas inimigas,da contra-inteligência dos inimigos,vencer a guerra pelo seu país,A
1950,1950,Kai contou o motivo do acidente no lago no últ...,Como descreveria Kai?,mais à vontade,como eles tiraram algo do peito,sentir- se triste,B
1951,1951,Kai melhorou a imagem de Jan e ela amava como ...,O que queremos Jan?,quis ser útil,quis a Jan para ser feliz,quadro da figura,C
1952,1952,Os filhos de Tracy queriam sorvete para que Au...,O que é necessário para o Aubrey?,contaram às crianças que responderam ao agrade...,gele sorvete,Agradecimentos à Aubrey,B


In [32]:
# Save to file
df_dev_pt.to_csv('translated/nllb_1.3b/dev_pt.csv')

### Translate testing base

In [33]:
df_tst_pt = df_tst.copy()
df_tst_pt.loc[:, ['context', 'question', 'answerA', 'answerB', 'answerC']] = None

translate_base(df_tst, df_tst_pt, model)

df_tst_pt.to_csv('translated/nllb_1.3b/tst_pt.csv')

100%|██████████| 2224/2224 [22:36<00:00,  1.64it/s]


### And the training base.

In [36]:
df_train_pt = df_train.copy()
df_train_pt.loc[:, ['context', 'question', 'answerA', 'answerB', 'answerC']] = None

translate_base(df_train, df_train_pt, model)

df_train_pt.to_csv('translated/nllb_1.3b/train_pt.csv')

100%|██████████| 33410/33410 [5:37:54<00:00,  1.65it/s]  
